# Get emails from Gmail API 

##### This is the quickstart code with a few mods

In [0]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().messages().list(userId='me').execute()
    messages = results.get('messages', [])

    if not messages:
        print('No labels found.')
    else:
        print('Labels:')
        for message in messages:
            print(message)

    return results, messages

### Separated out the steps to be able to play around

In [0]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=59513242107-tvsuj7o7ceiorkdsjp5d0jn9edib9pu3.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A45591%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=ALERwCZmmrOwP7g1ymXkAwwWLPaK56&access_type=offline


KeyboardInterrupt: ignored

In [0]:
with open('token.pickle', 'rb') as token:
    creds = pickle.load(token)

In [0]:
service = build('gmail', 'v1', credentials=creds)

In [0]:
results = service.users().messages().list(userId='me').execute()

### Dont remember where this is from

##### but it is two functions to get messages given an id

In [0]:
"""Get Message with given ID.
"""

import base64
import email
from apiclient import errors

def GetMessage(service, user_id, msg_id):
  """Get a Message with given ID.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    msg_id: The ID of the Message required.

  Returns:
    A Message.
  """
  try:
    message = service.users().messages().get(userId=user_id, id=msg_id).execute()

    print(f"Message snippet: {message['snippet']}") 

    return message
  except errors.HttpError as error:
    print(f'An error occurred: {error}')


def GetMimeMessage(service, user_id, msg_id):
  """Get a Message and use it to create a MIME Message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    msg_id: The ID of the Message required.

  Returns:
    A MIME Message, consisting of data from Message.
  """
  try:
    message = service.users().messages().get(userId=user_id, id=msg_id,
                                             format='raw').execute()

    print(f"Message snippet: {message['snippet']}")

    msg_str = base64.urlsafe_b64decode(message['raw'].encode('ASCII'))

    mime_msg = email.message_from_string(msg_str)

    return mime_msg
  except errors.HttpError as error:
    print(f'An error occurred: {error}')

##### breaking out the steps to figure out how it wokrs

In [0]:
one = GetMessage(service, 'me', results['messages'][0]['id'])

Message snippet: Your Indie Hackers community digest for November 12th ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌


In [0]:
mess = GetMessage(service, 'me', results['messages'][1]['id'])
Id = mess['id']
label_ids = mess['labelIds']
snippet = mess['snippet']
mime_type = mess['payload']['mimeType']
for item in mess['payload']['headers']:
  if item['name'] == 'From':
    from_ = item['value']
    if item['name'] == 'subject':
      subject = item['value']
print(Id, label_ids, from_, subject, snippet, mime_type)


In [0]:


two = GetMessage(service, 'me', results['messages'][1]['id'])

Message snippet: Sponsored by Barron&#39;s The first step toward taking control of your finances is understanding the landscape—and it helps to have a guide like Barron&#39;s. Today, we&#39;re sharing a collection of


In [0]:
for key in two.keys():
  print(key, two[key])

In [0]:
for item in two['payload']['headers']:
  if item['name'] == 'From':
    print(item['value'])

Pocket <noreply@getpocket.com>


In [0]:
len(results['messages'])

100

### Here i looped through the ids and wrote some information to the a csv

In [0]:
import csv

csv_file = open('gmail_api_emails.csv', 'w', encoding='UTF-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['id', 'label_ids', 'from_' 'snippet', 'mime_type'])
for msg in results['messages']:
  mess = GetMessage(service, 'me', msg['id'])
  Id = mess['id']
  label_ids = mess['labelIds']
  snippet = mess['snippet']
  mime_type = mess['payload']['mimeType']
  for item in mess['payload']['headers']:
    if item['name'] == 'From':
      from_ = item['value']
  
  csv_writer.writerow([Id, label_ids, from_, snippet, mime_type])

In [0]:
import pandas as pd

df = pd.read_csv('/content/gmail_api_emails.csv')

print(df.shape)

df.head()

In [0]:
flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

In [0]:
messages

In [0]:
dir()